In [0]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer
from sklearn import metrics
from sklearn.metrics import accuracy_score
import pandas as pd
import numpy as np

In [6]:

colnames = ['label', 'id', 'date', 'query','user','text']
df_train = pd.read_csv('training.1600000.processed.noemoticon.csv',
                      header=None, names=colnames, encoding='windows-1252')
df_test = pd.read_csv('testdata.manual.2009.06.14.csv',
                      header=None, names=colnames, encoding='windows-1252')
df_train.shape
df_train.head()
df_train['label'].value_counts()

4    800000
0    800000
Name: label, dtype: int64

In [8]:
bowVect = CountVectorizer()
trainBow = bowVect.fit_transform(df_train.text)
trainBow.shape

(1600000, 685256)

In [10]:

testBow = bowVect.transform(df_test[df_test.label!=2].text)
mnbClf = MultinomialNB().fit(trainBow, df_train['label'])
predicted = mnbClf.predict(testBow)

y_test = df_test[df_test.label != 2]

print(metrics.classification_report(y_test['label'],predicted))
print(accuracy_score(y_test['label'],predicted))

             precision    recall  f1-score   support

          0       0.79      0.82      0.81       177
          4       0.82      0.79      0.81       182

avg / total       0.81      0.81      0.81       359

0.807799442896936


In [20]:
x_train, x_test, y_train, y_test = train_test_split(df_train, df_train['label'], test_size=0.4)
getTweetCol = FunctionTransformer(lambda x: x['text'], validate = False)
tfVect= CountVectorizer(ngram_range=(1, 3), stop_words = "english")
mnb = MultinomialNB()
clf_tf = Pipeline([('getTweets',getTweetCol),('vect',tfVect),('clf',mnbClf)])
clf_tf.fit(x_train,y_train)
predicted = clf_tf.predict(x_test)

print(metrics.classification_report(y_test, predicted))
print(accuracy_score(y_test, predicted))

             precision    recall  f1-score   support

          0       0.76      0.80      0.78    319697
          4       0.79      0.75      0.77    320303

avg / total       0.77      0.77      0.77    640000

0.774303125
